# Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd
from random import randint as rint
from time import sleep

#### HH: мин/макс з/п, валюта

In [2]:
def get_hh_compen(item):
    try:
        vac_compen = item.find('div', {'class':'vacancy-serp-item__compensation'}).getText()
    except:
        vac_compen = None
    if vac_compen != None:
        vac_compen = vac_compen.split(' ')
        comp_curr = vac_compen[-1]
        if vac_compen[0] == 'от':
            min_compen = ''.join(vac_compen[1:-1])
            max_compen = None
        elif vac_compen[0] == 'до':
            max_compen = ''.join(vac_compen[1:-1])
            min_compen = None
        else:
            min_compen = vac_compen[0].split('-')[0]
            max_compen = vac_compen[0].split('-')[1]   
    else:
        min_compen, max_compen, comp_curr = [None] * 3
    return min_compen, max_compen, comp_curr

#### SJ: мин/макс з/п, валюта

In [3]:
def get_sj_compen(item):
    try:
        vac_compen = item.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText()
    except:
        vac_compen = None
    if vac_compen != None:
        if vac_compen == 'По договорённости':
            min_compen = None
            max_compen = None
            comp_curr = None
        elif '—' in vac_compen:
            vac_compen = vac_compen.split('—')
            comp_curr = vac_compen[1][-1]
            max_compen = vac_compen[1][:-1]
            min_compen = vac_compen[0]
        elif 'от' in vac_compen:
            vac_compen = vac_compen.split()
            comp_curr = vac_compen[-1]
            max_compen = None
            min_compen = ' '.join(vac_compen[1:-1])
        elif 'до' in vac_compen:
            vac_compen = vac_compen.split()
            comp_curr = vac_compen[-1]
            max_compen = ' '.join(vac_compen[1:-1])
            min_compen = None
        else:
            vac_compen = vac_compen.split()
            comp_curr = vac_compen[-1]
            max_compen = ' '.join(vac_compen[:-1])
            min_compen = ' '.join(vac_compen[:-1])
    else:
        min_compen, max_compen, comp_curr = [None] * 3
    return min_compen, max_compen, comp_curr

#### НН: прочие данные

In [4]:
def get_hh_data(vacancies):
    for item in vacancies:
        site_name = hh
        try:
            vac_name = item.find('a').getText()
        except:
            vac_name = None
        try:    
            vac_link = item.find('a')['href']
        except:
            vac_link = None
        try:
            employer = item.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).getText()
        except:
            employer = None
        try:    
            location = item.find('span', {'data-qa':'vacancy-serp__vacancy-address'}).getText()
        except:
            location = None
        min_compen, max_compen, comp_curr = get_hh_compen(item)
        df.loc[len(df) + 1] = [vac_name, employer, location, min_compen, max_compen, 
                                comp_curr, vac_link, site_name]

#### SJ: прочие данные

In [5]:
def get_sj_data(vacancies):
    for item in vacancies:
        site_name = sj
        try:
            vacancy = item.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
            vac_name = vacancy.getText()
        except:
            vac_name = None
        try:
            vac_link = sj + vacancy.findParent('a')['href']
        except:
            vac_link = None
        try:
            employer = item.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI'}).getText()          
        except:
            employer = None
        try:
            location = item.find('span', {'class':'_3mfro f-test-text-company-item-location _9fXTd _2JVkc _3e53o'}).findChildren(limit=2)[1].getText()
        except:
            location = None
        min_compen, max_compen, comp_curr = get_sj_compen(item)
        df.loc[len(df) + 1] = [vac_name, employer, location, min_compen, max_compen, 
                                comp_curr, vac_link, site_name]

In [6]:
df = pd.DataFrame(columns=['вакансия', 'компания', 'где', 'мин. з/п', 'макс. з/п',
                          'валюта', 'ссылка', 'сайт'])

In [7]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) \
Chrome/78.0.3904.97 Safari/537.36'
headers = {'User-Agent': user_agent}
hh = 'https://hh.ru'
sj = 'https://www.superjob.ru'

In [8]:
search_text = input('Наименование позиции ')

Наименование позиции Экономист


In [9]:
response = req.get(hh + f'/search/vacancy?text={search_text}', headers=headers)
if response.ok:
    hh_parsed_html = bs(response.text,'lxml')
    try:
        hh_vacancies_total = hh_parsed_html.find('div', {'class':'breadcrumbs'}).next_sibling.getText()
    except:
        hh_vacancies_total = 0
    try:    
        hh_pages_total = int(hh_parsed_html.find_all('a', {'data-qa':'pager-page'})[-1].getText())
    except:
        hh_pages_total = 0

In [10]:
print(f'на {hh} найдено {hh_vacancies_total}. всего страниц {hh_pages_total}')
input_pages = input('Сколько страниц смотреть ')
try:
    hh_pages = int(input_pages)
except:
    hh_pages = 0
if hh_pages > hh_pages_total:
    hh_pages = hh_pages_total
elif hh_pages < 1:
        hh_pages = 0

на https://hh.ru найдено 3 488 вакансий «Экономист». всего страниц 100
Сколько страниц смотреть 1


In [11]:
response = req.get(sj + f'/vacancy/search/?keywords={search_text}&geo%5Bc%5D%5B0%5D=1', headers=headers)
if response.ok:
    sj_parsed_html = bs(response.text,'lxml')
    try:
        sj_vacancies_total = sj_parsed_html.find('div', {'class':'_1tH7S _1o0Xp GPKTZ _3achh _3ofxL _2_FIo'}).findChild().getText()
    except:
        sj_vacancies_total = 0
    if ('Вакансий не найдено' in sj_vacancies_total) or (sj_vacancies_total == 0):
        sj_pages_total = 0
    else:
        try:
            sj_pages_total = int(sj_parsed_html.find_all('span', {'class':'qTHqo _2h9me DYJ1Y _2FQ5q _2GT-y'})[-2].getText())
        except:
            sj_pages_total = 1

In [12]:
print(f'на {sj} {sj_vacancies_total} "{search_text}". всего страниц {sj_pages_total}')
input_pages = input('Сколько страниц смотреть ')
try:
    sj_pages = int(input_pages)
except:
    sj_pages = 0
if sj_pages > sj_pages_total:
        sj_pages = sj_pages_total
elif sj_pages < 1:
        sj_pages = 0

на https://www.superjob.ru Найдено 626 вакансий "Экономист". всего страниц 16
Сколько страниц смотреть 1


In [13]:
for page in range(hh_pages):
    response = req.get(hh + f'/search/vacancy?text={search_text}&page={page}', headers=headers)
    if response.status_code == 200:
        hh_parsed_html = bs(response.text,'lxml')
        hh_vacancies_block = hh_parsed_html.find('div', {'class':'vacancy-serp'})
        hh_vacancies = hh_vacancies_block.findChildren('div', {'class':'vacancy-serp-item'}, recursive=False)
        get_hh_data(hh_vacancies)
    else:
        hh_err_log[page + 1] = response.status_code
        continue  
    sleep(rint(1, 5))

In [14]:
for page in range(sj_pages):
    response = req.get(sj + f'/vacancy/search/?keywords={search_text}&geo%5Bc%5D%5B0%5D=1&page={page + 1}', 
                   headers=headers)
    if response.status_code == 200:
        sj_parsed_html = bs(response.text,'lxml')
        sj_vacancies = sj_parsed_html.find_all('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})
        get_sj_data(sj_vacancies)
    else:
        sj_err_log[page + 1] = response.status_code
        continue
    sleep(rint(1, 5))

In [15]:
df

,вакансия,компания,где,мин. з/п,макс. з/п,валюта,ссылка,сайт
1,Главный экономист (контроллинг),Лодэ,Минск,4 000,4 000,бел. руб.,https://spb.hh.ru/vacancy/34876083?query=%D0%A...,https://hh.ru
2,Экономист по себестоимости,"Теремок, Группа компаний","Москва, Парк культуры",75 000,85 000,руб.,https://spb.hh.ru/vacancy/34767589?query=%D0%A...,https://hh.ru
3,Экономист-аналитик (бюджетирование),ООО Доминоспицца,Минск,1 000,1 500,бел. руб.,https://spb.hh.ru/vacancy/34074230?query=%D0%A...,https://hh.ru
4,Экономист отдела анализа кредитного риска,ЗАО Идея Банк,Минск,None,None,None,https://spb.hh.ru/vacancy/34419193?query=%D0%A...,https://hh.ru
5,Экономист Центра банковских услуг,ЗАО Идея Банк,Брест,None,None,None,https://spb.hh.ru/vacancy/34600257?query=%D0%A...,https://hh.ru
6,Бухгалтер-экономист,Молочный гостинец,"Минск, Могилевская",680,800,бел. руб.,https://spb.hh.ru/vacancy/35058588?query=%D0%A...,https://hh.ru
7,Материальный бухгалтер,ООО KAZ Minerals,Бишкек,None,None,None,https://spb.hh.ru/vacancy/35030380?query=%D0%A...,https://hh.ru
8,Ведущий экономист,ООО Проектсервис,"Санкт-Петербург, Московская",100 000,110 000,руб.,https://spb.hh.ru/vacancy/34744069?query=%D0%A...,https://hh.ru
9,Ведущий экономист,ООО УК Автодор,"Москва, Тверская и еще 2",120 000,120 000,руб.,https://spb.hh.ru/vacancy/34745523?query=%D0%A...,https://hh.ru
10,Экономист,АО Группа Синара,Екатеринбург,None,None,None,https://spb.hh.ru/vacancy/35075463?query=%D0%A...,https://hh.ru
